### Shared Word

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Spanish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
lang_pair = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 28  # 28  # native word end index

# word sample
word_sample = True  # True, False
word_sample_num = 20

# sentence check
twogram_sentence_check = False  # True, False
threegram_sentence_check = False
fourgram_sentence_check = False
fivegram_sentence_check = False

# shared word frequency
shared_word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word
etymology_word = True  # True for adding etymology word

# adding output file extention
if (not prefix_suffix) & etymology_word & native_word:
    file_ext = "1"
elif (not prefix_suffix) & etymology_word & (not native_word):
    file_ext = "2"
elif prefix_suffix & etymology_word & native_word:
    file_ext = "3"
elif prefix_suffix & etymology_word & (not native_word):
    file_ext = "4"
elif prefix_suffix & (not etymology_word) & native_word:
    file_ext = "5"
elif (not prefix_suffix) & (not etymology_word) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              
# 1 => for native word and etymology word without prefix suffix. 
# 2 => for only etymology word without prefix suffix. 
# 3 => for native word and etymology word with prefix suffix. prefix_suffix, native_word and etymology_word must be True. 
# 4 => for only etymology word with prefix suffix.
# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"{file_ext}")

1


In [4]:
Path(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/4-Shared Word/{lang_folder.capitalize()} {lang_pair.capitalize()}").mkdir(parents=True, exist_ok=True)

In [5]:
def remove_repetition(word_group):
    '''
    remove_repetition(word_group): detect word repetion in word group 
    '''
    words = word_tokenize(word_group)
    word_unique = set(words)
    if len(word_unique) == 1:
        return "repetitive_word_group"
    else:
        return word_group

In [6]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,que,37853284
1,de,37809537
2,no,33043466
3,a,25439588
4,la,24024343
...,...,...
446459,bibe,5
446460,dejugo,5
446461,bibbi,5
446462,bibberman,5


In [7]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [8]:
# Option
if prefix_suffix:
    df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
    df_word = df_word.loc[:,["word","frequency"]]
    df_word = pd.concat([df_word,df_word_select], axis=0)
    df_word.drop_duplicates(inplace=True)    
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    df_word = df_word_select

if native_word:
    df_word
else:
    df_word = df_word.head(0)

df_word

,word,frequency
0,que,37853284
1,de,37809537
2,no,33043466
3,a,25439588
4,la,24024343
5,el,19975672
6,es,18510392
7,y,18426158
8,en,17819385
9,lo,16131539


In [9]:
#lang_pair_list = glob.glob(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()}_And_{lang_pair.lower().capitalize()}*_All.xlsx")
#lang_pair_list

In [10]:
#df_pair = pd.read_excel(f"{lang_pair_list[0]}")
df_pair = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.lower().capitalize()}/{lang_folder.capitalize()}_{lang_pair.lower().capitalize()}_Shared_Vocabulary.xlsx")
#df_pair = df_pair.head()
df_pair

,dict_entry_main,turkish_word
0,grupo,grup
1,masaje,masaj
2,oxígeno,oksijen
3,bicicleta,bisiklet
4,ducha,duş
...,...,...
1525,cebra,zebra
1526,zigzag,zikzak
1527,zombi,zombi
1528,zoología,zooloji


In [11]:
# Option
if prefix_suffix:
    df_prefix_suffix_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Word_Prefix_Suffix_Custom_Result.xlsx")
    df_prefix_suffix_select = df_prefix_suffix_select.loc[:,["search_word","word"]]
    df_prefix_suffix_select.rename(columns={"search_word":"dict_entry_main"}, inplace=True)
    df_pair = pd.merge(df_pair,df_prefix_suffix_select, how="inner", on="dict_entry_main")
    df_pair.drop_duplicates(inplace=True)
    df_pair.reset_index(drop=True, inplace=True)
    df_pair = df_pair.loc[:,["word",f"{lang_pair.lower()}_word"]]
    df_pair.rename(columns={"word":"dict_entry_main"}, inplace=True)
else:
    pass

if etymology_word:
    df_pair
else:
    df_pair = df_pair.head(0)
    
df_pair

,dict_entry_main,turkish_word
0,grupo,grup
1,masaje,masaj
2,oxígeno,oksijen
3,bicicleta,bisiklet
4,ducha,duş
...,...,...
1525,cebra,zebra
1526,zigzag,zikzak
1527,zombi,zombi
1528,zoología,zooloji


In [12]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [13]:
word_select = df_word["word"].values.tolist()
words = df_pair["dict_entry_main"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)
words_set = set(words)

In [14]:
word_list = list(word_select_set.union(words_set.difference(disable_word_set)))

In [15]:
# 2 way  Not: 1 way is long and in Shared Word ReDe part 
data_kind_list = ["Two", "Three", "Four", "Five", "Sentence"]
for i in data_kind_list:
    sent_ngram_file = f"{i}" # Two, Three, Four, Five, Sentence
 
    if sent_ngram_file.lower() != "sentence":
        data_kind = f"{sent_ngram_file}gram"  # Twogram, Threegram, Fourgram, Fivegram
        folder_path = "N Gram"
    else:
        data_kind = "sentence"
        folder_path = "Sentence"

    df = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/{folder_path.capitalize()}/Merge/{data_kind.capitalize()}_Merge.csv")
    d_list  = df.iloc[:,0].values.tolist()

    resultlist = []
    manager = multiprocessing.Manager()
    resultlist = manager.list()
    
    def word_in_wordgroup(d_list):
        mergelist = []
        try:
            word = d_list.split()
        except:
            word = []
            #pass  disabled for non split value
        var1 = range(len(word))
        for j in var1:
            if word[j] in word_list:
                mergelist.append(word[j])
                if len(mergelist) == len(word):
                        resultlist.append(d_list)
                            
    if __name__ == '__main__':
        # with Pool(16) as p:
        with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
            p.map(word_in_wordgroup, d_list) # string_word liste

    result_list = list(resultlist)
    df_result = pd.DataFrame(result_list, columns=[0])  # add columns parameter for empty result
    df_result = df_result.rename(columns = {0: f"{data_kind.lower()}"})
    df_merge = pd.merge(df_result, df, how="left", on=f"{data_kind.lower()}")
    df_merge_result = df_merge.sort_values(by="frequency", ascending=False)
    df_merge_result.drop_duplicates(inplace=True)
    df_merge_result[f"{data_kind.lower()}"] = df_merge_result[f"{data_kind.lower()}"].apply(lambda x: remove_repetition(x))  # added remove_repetition 
    df_merge_result = df_merge_result[df_merge_result[f"{data_kind.lower()}"] != "repetitive_word_group"]  # remove_repetition part
    df_merge_result.reset_index(drop=True, inplace=True)

    df_merge_result.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_{data_kind.capitalize()}{file_ext}.xlsx", index=False)    

#### Concat Result

In [16]:
df_word = pd.DataFrame(word_list, columns=["word"])
df_word

,word
0,fiasco
1,trauma
2,terapeuta
3,diesel
4,revisión
...,...
1519,astronauta
1520,buffet
1521,riesgo
1522,dosis


In [17]:
# Option For Frequency
if shared_word_frequency:
    df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
    df_word = pd.merge(df_word,df_word_all, how="inner", on="word")
    df_word.drop_duplicates(inplace=True)
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    pass

df_word

,word,frequency
0,que,37853284
1,de,37809537
2,no,33043466
3,a,25439588
4,la,24024343
...,...,...
1490,itálico,5
1491,clorado,5
1492,ortopedía,5
1493,aerobio,5


In [18]:
df_shared_twogram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Twogram{file_ext}.xlsx")
df_shared_twogram = df_shared_twogram.loc[:,["twogram","frequency"]]
df_shared_twogram.rename(columns={"frequency":"freq_twogram"}, inplace=True)
df_shared_twogram

,twogram,freq_twogram
0,lo que,1485227
1,por qué,1460656
2,no lo,1252184
3,no es,1114995
4,está bien,963477
...,...,...
44983,ritmo bien,3
44984,femenino se,3
44985,magnetismo por,3
44986,magnetismo para,3


In [19]:
# Option For Twogram Sentence
if twogram_sentence_check:
    df_twogram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Two_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_twogram_sent.rename(columns={"two_gram":"twogram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_twogram_sent = df_twogram_sent.loc[:,["twogram"]]
    df_shared_twogram = pd.merge(df_shared_twogram, df_twogram_sent, how="inner", on="twogram")
    df_shared_twogram.drop_duplicates(inplace=True)
    df_shared_twogram.reset_index(drop=True, inplace=True)
else:
    pass

df_shared_twogram


,twogram,freq_twogram
0,lo que,1485227
1,por qué,1460656
2,no lo,1252184
3,no es,1114995
4,está bien,963477
...,...,...
44983,ritmo bien,3
44984,femenino se,3
44985,magnetismo por,3
44986,magnetismo para,3


In [20]:
df_shared_threegram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Threegram{file_ext}.xlsx")
df_shared_threegram = df_shared_threegram.loc[:,["threegram","frequency"]]
df_shared_threegram.rename(columns={"frequency":"freq_threegram"}, inplace=True)
df_shared_threegram

,threegram,freq_threegram
0,por qué no,344733
1,es lo que,296245
2,eso es lo,113501
3,qué es eso,109973
4,eso no es,92222
...,...,...
121577,es su chip,3
121578,solo con terror,3
121579,de dinosaurio de,3
121580,de dinosaurio y,3


In [21]:
# Option For Threegram Sentence
if threegram_sentence_check:
    df_threegram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Three_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_threegram_sent.rename(columns={"three_gram":"threegram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_threegram_sent = df_twogram_sent.loc[:,["threegram"]]
    df_shared_threegram = pd.merge(df_shared_threegram, df_threegram_sent, how="inner", on="threegram")
    df_shared_threegram.drop_duplicates(inplace=True)
    df_shared_threegram.reset_index(drop=True, inplace=True)
else:
    pass

df_shared_threegram

,threegram,freq_threegram
0,por qué no,344733
1,es lo que,296245
2,eso es lo,113501
3,qué es eso,109973
4,eso no es,92222
...,...,...
121577,es su chip,3
121578,solo con terror,3
121579,de dinosaurio de,3
121580,de dinosaurio y,3


In [22]:
df_shared_fourgram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Fourgram{file_ext}.xlsx")
df_shared_fourgram = df_shared_fourgram.loc[:,["fourgram","frequency"]]
df_shared_fourgram.rename(columns={"frequency":"freq_fourgram"}, inplace=True)
df_shared_fourgram

,fourgram,freq_fourgram
0,eso es lo que,101130
1,qué es lo que,60509
2,por qué no me,39905
3,no es lo que,28512
4,por qué no lo,22222
...,...,...
107739,el párrafo de la,3
107740,el pánico por la,3
107741,sistema central que los,3
107742,el pánico pero lo,3


In [23]:
# Option For Fourgram Sentence
if fourgram_sentence_check:
    df_fourgram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Four_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_fourgram_sent.rename(columns={"four_gram":"fourgram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_fourgram_sent = df_twogram_sent.loc[:,["fourgram"]]
    df_shared_fourgram = pd.merge(df_shared_fourgram, df_fourgram_sent, how="inner", on="fourgram")
    df_shared_fourgram.drop_duplicates(inplace=True)
    df_shared_fourgram.reset_index(drop=True, inplace=True)
else:
    pass

df_shared_fourgram

,fourgram,freq_fourgram
0,eso es lo que,101130
1,qué es lo que,60509
2,por qué no me,39905
3,no es lo que,28512
4,por qué no lo,22222
...,...,...
107739,el párrafo de la,3
107740,el pánico por la,3
107741,sistema central que los,3
107742,el pánico pero lo,3


In [24]:
df_shared_fivegram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Fivegram{file_ext}.xlsx")
df_shared_fivegram = df_shared_fivegram.loc[:,["fivegram","frequency"]]
df_shared_fivegram.rename(columns={"frequency":"freq_fivegram"}, inplace=True)
df_shared_fivegram

,fivegram,freq_fivegram
0,por qué no me lo,11245
1,eso no es lo que,8546
2,eso es lo que me,6031
3,qué es lo que te,4002
4,y eso es lo que,3529
...,...,...
46146,sí pero en una misión,3
46147,y el médico de guardia,3
46148,un botón de un uniforme,3
46149,sí pero en teoría sí,3


In [25]:
# Option For Fivegram Sentence
if fivegram_sentence_check:
    df_fivegram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Five_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_fivegram_sent.rename(columns={"five_gram":"fivegram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_fivegram_sent = df_twogram_sent.loc[:,["fivegram"]]
    df_shared_fivegram = pd.merge(df_shared_fivegram, df_fivegram_sent, how="inner", on="fivegram")
    df_shared_fivegram.drop_duplicates(inplace=True)
    df_shared_fivegram.reset_index(drop=True, inplace=True)
else:
    pass

df_shared_fivegram

,fivegram,freq_fivegram
0,por qué no me lo,11245
1,eso no es lo que,8546
2,eso es lo que me,6031
3,qué es lo que te,4002
4,y eso es lo que,3529
...,...,...
46146,sí pero en una misión,3
46147,y el médico de guardia,3
46148,un botón de un uniforme,3
46149,sí pero en teoría sí,3


In [26]:
df_shared_sentence = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Sentence{file_ext}.xlsx")
df_shared_sentence = df_shared_sentence.loc[:,["sentence","frequency"]]
df_shared_sentence.rename(columns={"frequency":"freq_sentence"}, inplace=True)
df_shared_sentence

,sentence,freq_sentence
0,está bien,506251
1,de acuerdo,344687
2,por qué,330492
3,qué es eso,88217
4,eso es,87824
...,...,...
27699,y por eso cobra,3
27700,mi señora es un honor,3
27701,qué es terminal,3
27702,sí sí en parte,3


In [27]:
df_shared_all = pd.concat([df_word, df_shared_twogram, df_shared_threegram, df_shared_fourgram, df_shared_fivegram, df_shared_sentence], axis=1)
df_shared_all

,word,frequency,twogram,freq_twogram,threegram,freq_threegram,fourgram,freq_fourgram,fivegram,freq_fivegram,sentence,freq_sentence
0,que,37853284.0,lo que,1485227.0,por qué no,344733,eso es lo que,101130.0,por qué no me lo,11245.0,está bien,506251.0
1,de,37809537.0,por qué,1460656.0,es lo que,296245,qué es lo que,60509.0,eso no es lo que,8546.0,de acuerdo,344687.0
2,no,33043466.0,no lo,1252184.0,eso es lo,113501,por qué no me,39905.0,eso es lo que me,6031.0,por qué,330492.0
3,a,25439588.0,no es,1114995.0,qué es eso,109973,no es lo que,28512.0,qué es lo que te,4002.0,qué es eso,88217.0
4,la,24024343.0,está bien,963477.0,eso no es,92222,por qué no lo,22222.0,y eso es lo que,3529.0,eso es,87824.0
...,...,...,...,...,...,...,...,...,...,...,...,...
121577,NaN,NaN,NaN,NaN,es su chip,3,NaN,NaN,NaN,NaN,NaN,NaN
121578,NaN,NaN,NaN,NaN,solo con terror,3,NaN,NaN,NaN,NaN,NaN,NaN
121579,NaN,NaN,NaN,NaN,de dinosaurio de,3,NaN,NaN,NaN,NaN,NaN,NaN
121580,NaN,NaN,NaN,NaN,de dinosaurio y,3,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
df_shared_all.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Result_With_Frequency{file_ext}.xlsx", index=False)

##### Concat Result With Comma

In [29]:
def word_in_wordgroup(df, source_column, target_column):

    '''word_in_wordgroup(df, "word", "twogram"):
       df is dataframe, source_column and target_column are 
       dataframe column string name. source_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_word_result = pd.DataFrame()
    for i in df[f"{source_column}"].dropna():
        try:
            if word_sample:
                word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)  # Option
            else:
                word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)] 
        except:
            pass        
        word_in_word_cluster.insert(0,f"{source_column}",i)
        df_word_result = pd.concat([df_word_result,word_in_word_cluster], axis=0)
    df_word_result.reset_index(drop=True, inplace=True)

    return df_word_result

In [30]:
df_word_order_twogram = word_in_wordgroup(df_shared_all, "word", "twogram")
df_word_order_threegram = word_in_wordgroup(df_shared_all, "word", "threegram") 
df_word_order_fourgram = word_in_wordgroup(df_shared_all, "word", "fourgram") 
df_word_order_fivegram = word_in_wordgroup(df_shared_all, "word", "fivegram")
df_word_order_sentence = word_in_wordgroup(df_shared_all, "word", "sentence") 

In [31]:
df_word_order_join_twogram = df_word_order_twogram.groupby(["word"])["twogram"].apply(", ".join).reset_index()   # df_word_order_11.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].transform(lambda x: ','.join(x))
df_word_order_join_threegram = df_word_order_threegram.groupby(["word"])["threegram"].apply(", ".join).reset_index()
df_word_order_join_fourgram = df_word_order_fourgram.groupby(["word"])["fourgram"].apply(", ".join).reset_index()
df_word_order_join_fivegram = df_word_order_fivegram.groupby(["word"])["fivegram"].apply(", ".join).reset_index()
df_word_order_join_sentence = df_word_order_sentence.groupby(["word"])["sentence"].apply(", ".join).reset_index()

In [32]:
dfs = [df_word_order_join_twogram,df_word_order_join_threegram,df_word_order_join_fourgram,df_word_order_join_fivegram,df_word_order_join_sentence]

In [33]:
df_word_order_join_all = reduce(lambda  left,right: pd.merge(left,right, on=['word'], how='outer'), dfs)  # left,right make left to right merge
#df_word_order_join_all = reduce(lambda  right,left: pd.merge(left,right, on=['word'], how='outer'), dfs)  # right,left make right to left merge
df_word_order_join_all  

,word,twogram,threegram,fourgram,fivegram,sentence
0,a,"a la, a mi, a los, a un, a su, a qué, a una, y...","a la policía, a qué te, a punto de, a lo que, ...","a lo que me, está a punto de, es a lo que, a l...","es a lo que me, eso es a lo que, no es a lo qu...","a qué, a la carga, a mi, a que sí, a la policí..."
1,abogado,"un abogado, mi abogado, el abogado, su abogado...","a un abogado, a mi abogado, el abogado de, con...","el abogado de la, es el abogado de, con el abo...","el abogado de la familia, abogado con una cris...","un abogado, es abogado, el abogado, mi abogado..."
2,absorber,"para absorber, absorber los, que absorber, abs...","que absorber los, de absorber energía, para ab...","es para absorber la, mi cuerpo a absorber, cue...","a mi cuerpo a absorber, a absorber el suero y,...",bien absorber
3,absurdo,"es absurdo, qué absurdo, absurdo que, absurdo ...","eso es absurdo, no es absurdo, es absurdo no, ...","pero eso es absurdo, no eso es absurdo, eso es...","lo absurdo de lo que, absurdo en la forma en, ...","es absurdo, eso es absurdo, qué absurdo, pero ..."
4,acacia,"la acacia, acacia en, acacia que, de acacia, y...","la acacia que, jazmín y acacia, en una acacia,...",de jazmín y acacia,el de jazmín y acacia,el de jazmín y acacia
...,...,...,...,...,...,...
1474,índice,"índice de, el índice, un índice, su índice, mi...","el índice de, en el índice, un índice de, su í...","en el índice de, con el índice de, el índice q...","índice eso es lo que, simulación en el índice ...",el índice
1475,ópera,"la ópera, una ópera, de ópera, ópera de, ópera...","de la ópera, a la ópera, en la ópera, para la ...","teatro de la ópera, de la ópera de, en la óper...","el teatro de la ópera, mi teatro de la ópera, ...","la ópera, a la ópera, una ópera, en la ópera, ..."
1476,óptico,"efecto óptico, óptico y, óptico está, óptico d...","un efecto óptico, óptico y la, de camuflaje óp...","tecnología de camuflaje óptico, es un efecto ó...","la tecnología de camuflaje óptico, un equipo d...","un efecto óptico, es un efecto óptico, tecnolo..."
1477,óptimo,"es óptimo, está óptimo, nivel óptimo, óptimo c...","el nivel óptimo, nivel óptimo con, óptimo con ...","nivel óptimo con rotación, el nivel óptimo con...",el nivel óptimo con rotación,"está óptimo, eso es óptimo, que óptimo, eso no..."


In [34]:
# Option For Word Frequency
if shared_word_frequency:
    df_word_order_join_all = pd.merge(df_word_order_join_all,df_word_all, how="inner", on="word")
    df_word_order_join_all.drop_duplicates(inplace=True)
    df_word_order_join_all = df_word_order_join_all.loc[:,["word","frequency","twogram","threegram","fourgram","fivegram","sentence"]]
    df_word_order_join_all.sort_values(by="frequency", inplace=True, ascending=False)
    df_word_order_join_all.reset_index(drop=True, inplace=True)
else:
    pass

df_word_order_join_all

,word,frequency,twogram,threegram,fourgram,fivegram,sentence
0,que,37853284,"lo que, que no, que te, que me, que lo, que es...","es lo que, de lo que, lo que me, lo que te, qu...","eso es lo que, qué es lo que, no es lo que, es...","eso no es lo que, eso es lo que me, qué es lo ...","por que, es que, que es eso, que es, que bien,..."
1,de,37809537,"de la, de acuerdo, de qué, de los, de que, de ...","de lo que, a punto de, de qué se, de que no, n...","de acuerdo de acuerdo, está a punto de, una ta...","eso es de lo que, de eso es de lo, por el bien...","de acuerdo, de qué, sí de acuerdo, de acuerdo ..."
2,no,33043466,"no lo, no es, no me, que no, no te, no se, qué...","por qué no, eso no es, no no lo, no es un, no ...","por qué no me, no es lo que, por qué no lo, po...","por qué no me lo, eso no es lo que, no es eso ...","por qué no, no está bien, no lo se, no lo es, ..."
3,a,25439588,"a la, a mi, a los, a un, a su, a qué, a una, y...","a la policía, a qué te, a punto de, a lo que, ...","a lo que me, está a punto de, es a lo que, a l...","es a lo que me, eso es a lo que, no es a lo qu...","a qué, a la carga, a mi, a que sí, a la policí..."
4,la,24024343,"a la, de la, en la, es la, por la, que la, con...","a la policía, está en la, no es la, a la escue...","la forma en que, por qué no la, no está en la,...","no está en la lista, es la forma en que, por e...","la policía, a la carga, es la policía, la qué,..."
...,...,...,...,...,...,...,...
1474,briqueta,18,una briqueta,es una briqueta,club es una briqueta,NaN,NaN
1475,caroteno,14,"beta caroteno, caroteno y","beta caroteno y, de beta caroteno",de beta caroteno y,NaN,NaN
1476,rosetón,9,el rosetón,NaN,NaN,NaN,NaN
1477,aerobio,5,motor aerobio,NaN,NaN,NaN,NaN


In [35]:
df_word_order_join_all.word.nunique()

1479

In [36]:
df_word_order_join_all.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Join_Result_Without_Frequency{file_ext}.xlsx", index=False)

#### Copy Move And Delete

In [37]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared*{file_ext}.xlsx")
output_file

['Spanish_Turkish_Shared_Twogram1.xlsx',
 'Spanish_Turkish_Shared_Threegram1.xlsx',
 'Spanish_Turkish_Shared_Fourgram1.xlsx',
 'Spanish_Turkish_Shared_Fivegram1.xlsx',
 'Spanish_Turkish_Shared_Sentence1.xlsx',
 'Spanish_Turkish_Shared_Result_With_Frequency1.xlsx',
 'Spanish_Turkish_Shared_Join_Result_Without_Frequency1.xlsx']

In [38]:
for k in output_file:
    source = k # source directory
    destination = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/4-Shared Word/{lang_folder.capitalize()} {lang_pair.capitalize()}"
    shutil.copy2(source, destination)

In [39]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass